In [1]:
import torch
# Is CUDA available now?
print('CUDA available: {}'.format(torch.cuda.is_available()))

CUDA available: True


In [1]:
!conda env list

# conda environments:
#
base                     /home/chirathv97/anaconda3
yolo3dpy37            *  /home/chirathv97/anaconda3/envs/yolo3dpy37



In [1]:
import os
os.getcwd()

'/home/theekshana/Documents/visualDet3D'

In [45]:
cd ..

C:\Users\FYP\Documents\code\visualDet3D


In [2]:
!chmod 777 -R /home/theekshana/Documents/visualDet3D

In [52]:
!pip install -r requirement.txt

In [4]:
!./make.sh

~/Documents/visualDet3D/visualDet3D/networks/lib/ops/dcn ~/Documents/visualDet3D
running build_ext
building '..deform_conv_ext' extension
creating /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/dcn/build
creating /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/dcn/build/temp.linux-x86_64-cpython-37
creating /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/dcn/build/temp.linux-x86_64-cpython-37/src
creating /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/dcn/build/temp.linux-x86_64-cpython-37/src/cuda
Emitting ninja build file /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/dcn/build/temp.linux-x86_64-cpython-37/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/3] c++ -MMD -MF /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/dcn/build/temp.linux-x86_64-cpython-37/src/d

[2/2] c++ -MMD -MF /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/iou3d/build/temp.linux-x86_64-cpython-37/src/iou3d.o.d -pthread -B /home/theekshana/anaconda3/envs/YOLO3D/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fopenmp -I/home/theekshana/anaconda3/envs/YOLO3D/include -fPIC -DWITH_CUDA -I/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/torch/include -I/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/torch/include/torch/csrc/api/include -I/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/torch/include/TH -I/home/theekshana/anaconda3/envs/YOLO3D/lib/python3.7/site-packages/torch/include/THC -I/home/theekshana/anaconda3/envs/YOLO3D/include -I/home/theekshana/anaconda3/envs/YOLO3D/include/python3.7m -c -c /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/ops/iou3d/src/iou3d.cpp -o /home/theekshana/Documents/visualDet3D/visualDet3D/networks/lib/op

In [16]:
torch.version.cuda

'11.6'

In [54]:
!pip install coloredlogs

In [21]:
%pwd

'/home/theekshana/Documents/visualDet3D'

In [2]:
import sys
# sys.path.append("../")
import importlib
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
import cython
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, models, transforms
import torchvision
from numba import jit
from tqdm import tqdm

In [3]:
from visualDet3D.data.kitti.utils import write_result_to_file
from visualDet3D.utils.utils import LossLogger, cfg_from_file
from visualDet3D.networks.utils.registry import DETECTOR_DICT, DATASET_DICT, PIPELINE_DICT
from visualDet3D.networks.heads.anchors import Anchors
from visualDet3D.networks.lib.fast_utils.hill_climbing import post_opt
from visualDet3D.networks.utils import BBox3dProjector, BackProjection
from visualDet3D.utils.utils import convertAlpha2Rot, convertRot2Alpha, draw_3D_box, compound_annotation
import visualDet3D.data.kitti.dataset
from visualDet3D.utils.timer import Timer

In [4]:
print('CUDA available: {}'.format(torch.cuda.is_available()))

cfg = cfg_from_file("config/config.py")
is_test_train = True

checkpoint_name = "Stereo3D_latest.pth"

CUDA available: True


In [ ]:
#!./launchers/eval.sh config/config.py 0 $CHECKPOINT_PATH validation/test

### Compute image database and anchors mean/std

In [2]:
!./launchers/det_precompute.sh config/config.py train

Precomputation for the training/validation split
start reading training data
training split finished precomputing02s, eta:2.00s, total_objs:[20792, 3682], usable_objs:[19582, 3283]]
start reading validation data
validation split finished precomputings, eta:0.02s, total_objs:[0, 0], usable_objs:[0, 0]
Preprocessing finished


In [3]:
!./launchers/det_precompute.sh config/config.py test

Precomputation for the test split
start reading test data
test split finished precomputing00s, eta:0.01s
Preprocessing finished


In [ ]:
## run this if disparity map is needed, can be computed with point cloud or openCV BlockMatching
# You can run ./launcher/disparity_precompute.sh without arguments to see helper documents
!./disparity_precompute.sh config/config.py $IsUsingPointCloud

In [ ]:
## train the model with one GPU
# You can run ./launcher/train.sh without arguments to see helper documents
!./launcher/train.sh  config/config.py 0 $experiment_name # validation goes along

In [ ]:
## produce validation/test result # we only support single GPU testing
# You can run ./launcher/eval.sh without arguments to see helper documents
!./launcher/eval.sh -config/config.py 0 $CHECKPOINT_PATH validation/test